<a href="https://colab.research.google.com/github/LauraSchellekens/TM10007_-private_-group/blob/Lisanne_SVM/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git
!pip install sklearn numpy matplotlib

In [0]:
# General packages
import numpy as np 
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import impute
from sklearn.impute import SimpleImputer

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report


# import classifier SVC
from sklearn.svm import SVC


In [90]:
# Load data
from adni.load_data import load_data
data = load_data()

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
X = data.drop('label', axis=1) # All data without column 'label'
Y = data['label'] # AD en CN moeten alleen nog omgezet worden naar 0 en 1
Y.replace(('AD', 'CN'), (1, 0), inplace=True) # AD en CN omgezet naar 0 en 1
print(X.shape)
print(Y.shape)
print(X)


The number of samples: 855
The number of columns: 268
(855, 267)
(855,)
                     hf_energy  ...  vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0
ID                              ...                                         
0_002_S_0413_bl_0  5403.229408  ...                             4.207831e-10
0_002_S_0559_bl_0   922.459760  ...                             3.137258e-10
0_002_S_0619_bl_0  1674.765276  ...                             4.219847e-10
0_002_S_0685_bl_0  3443.797200  ...                             5.954654e-11
0_002_S_0816_bl_0   914.469846  ...                             4.174750e-10
...                        ...  ...                                      ...
0_941_S_4292_bl_0  1503.564446  ...                             2.792072e-10
0_941_S_4365_bl_0  1087.136508  ...                             3.406526e-10
0_941_S_4376_bl_0  1963.323949  ...                             1.294541e-10
0_941_S_5124_bl_0  2392.983035  ...                             1.737102e-10
0_94

In [91]:
data

,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,label,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_002_S_0413_bl_0,5403.229408,4.679203,2.392463,2.040475,1.375697,1.487644,-0.062859,429,0.364471,2.103334,-1.481394,0.455533,1,6395.491754,9940.032401,14622.897328,4.072445,5.204894,4.841280,3.774545,-0.108664,2.224809,0.498333,0.746542,0.785263,-0.007726,-0.069990,-0.029403,-0.000075,-0.068204,0.008103,-0.632451,-1.092715,-1.402496,1304,210,608,0.153782,0.627844,0.457846,...,0.020347,0.022628,0.000556,0.018271,2.374306e-14,4.104987,7.559967,6.673826e-09,1.741492e-09,1.299429e-09,1.734096e-10,514,1.519472e-09,6.500416e-09,2.307441,1.554520e-09,2.374306e-14,4.104987,7.559967,6.673826e-09,1.741492e-09,1.299429e-09,1.734096e-10,514,1.519472e-09,6.500416e-09,2.307441,1.554520e-09,2.446356e-17,3.969563,4.283295,1.930203e-09,4.762307e-10,3.147900e-10,1.195016e-10,8,4.503026e-10,1.810702e-09,2.003542,4.207831e-10
0_002_S_0559_bl_0,922.459760,4.772376,1.709731,1.437500,0.863074,0.940709,-0.247543,412,0.347258,1.685042,-1.222797,0.377850,1,8432.620063,5549.455518,12050.086848,4.076190,5.228020,4.830720,3.252139,-0.730229,0.348783,0.483188,0.412527,0.529344,-0.001275,-0.127039,-0.054140,0.000178,-0.145082,-0.044920,-0.578344,-0.633490,-0.828795,906,229,317,0.189966,0.455902,0.447878,...,0.010227,0.039959,0.001927,0.009183,6.260213e-15,4.471662,3.674709,2.910256e-09,8.867658e-10,7.190769e-10,4.342644e-11,421,8.276855e-10,2.866830e-09,1.605653,7.019460e-10,6.260213e-15,4.471662,3.674709,2.910256e-09,8.867658e-10,7.190769e-10,4.342644e-11,421,8.276855e-10,2.866830e-09,1.605653,7.019460e-10,2.058198e-17,4.401233,2.048344,1.189167e-09,3.678934e-10,3.325169e-10,6.933179e-13,11,4.3

In [92]:
# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(598, 267) (598,)
(257, 267) (257,)


In [93]:
# Preprocessing: drop feature if too many missing values

X_train = X_train.replace(0, np.NaN)                  # replace 0 with NaN
missing_per_feature = X_train.isnull().sum()          # gives the amount of missing values (NaN) per feature
pct_null = missing_per_feature / len(X_train)         # gives percentage of missing values per feature
missing_features = pct_null[pct_null > 0.40].index    # gives features with more than 40% missing values
X_train.drop(missing_features, axis=1, inplace=True)  # remove feature if more than 40% missing values

missing_per_feature_max = max(X_train.isnull().sum()) # gives the maximum amount of missing values (NaN) per feature AFTER dropping incomplete features

print(missing_per_feature_max)
print (X_train.shape, y_train.shape)

95
(598, 261) (598,)


In [94]:
# nu ook voor X_test
X_test = X_test.replace(0, np.NaN)                  # replace 0 with NaN
missing_per_feature = X_test.isnull().sum()          # gives the amount of missing values (NaN) per feature
pct_null = missing_per_feature / len(X_test)         # gives percentage of missing values per feature
missing_features = pct_null[pct_null > 0.40].index    # gives features with more than 40% missing values
X_test.drop(missing_features, axis=1, inplace=True)  # remove feature if more than 40% missing values

missing_per_feature_max = max(X_test.isnull().sum()) # gives the maximum amount of missing values (NaN) per feature AFTER dropping incomplete features

print(missing_per_feature_max)
print (X_test.shape, y_test.shape)

45
(257, 261) (257,)


In [95]:
X_train

,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,logf_range_sigma1,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_123_S_0094_bl_0,3386.506846,5.259573,-1.403587,2.015655,0.960593,1.216723,-0.316327,232,1.502769,2.331983,-0.331961,0.770091,13751.589551,4647.982362,8708.776471,3.968036,5.213308,5.118783,5.757122,-0.356722,-0.557490,0.580235,0.476846,0.559548,-0.001629,-0.151980,-0.090774,0.006213,-0.192424,-0.082944,-1.071293,-0.733741,-0.798981,1187,277,317,0.213793,0.426781,0.533687,1.651528,...,0.003134,0.142357,2.305241e-02,0.003097,7.101581e-14,4.627953,2.142850,1.105884e-08,2.898888e-09,1.856321e-09,2.180677e-10,426,2.923607e-09,1.084077e-08,1.575443,2.727167e-09,7.101581e-14,4.627953,2.142850,1.105884e-08,2.898888e-09,1.856321e-09,2.180677e-10,426,2.923607e-09,1.084077e-08,1.575443,2.727167e-09,5.750367e-16,4.317135e+00,4.456912,5.226646e-09,1.563623e-09,1.370837e-09,2.608806e-10,16,7.946750e-10,4.965765e-09,1.984802,1.164944e-09
0_067_S_0257_bl_0,1315.517401,4.622818,3.056713,1.954463,1.326876,1.366179,0.248353,159,0.381970,1.706110,-1.234939,0.374791,4779.811318,3114.233543,2644.678724,4.204420,5.032773,5.214473,4.097647,-0.922221,-0.137989,0.624921,0.512730,0.665366,-0.000942,-0.029024,-0.009665,0.004493,0.015073,0.011748,-0.949891,-0.714890,-0.875084,343,119,123,0.256804,0.652459,0.526512,1.574812,...,0.085414,0.012616,1.213121e-04,0.075915,1.049943e-14,4.741863,1.434665,5.728093e-09,1.895251e-09,1.495836e-09,1.861331e-10,147,1.887399e-09,5.541960e-09,1.216039,1.437026e-09,1.049943e-14,4.741863,1.434665,5.728093e-09,1.895251e-09,1.495836e-09,1.861331e-10,147,1.887399e-09,5.541960e-09,1.216039,1.437026e-09,3.501646e-18,2.321928e+00,-0.784326,6.958133e-10,4

In [96]:
# Preprocessing:  impute missing values (fill missing)

imputer = impute.SimpleImputer(strategy='mean')     # imputer with mean 
X_train_imp = imputer.fit_transform(X_train)          # impute  
X_train_imp = pd.DataFrame(data=X_train_imp, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df

type(X_train_imp)

# en nu ook voor x_test
imputer = impute.SimpleImputer(strategy='mean')     # imputer with mean 
X_test_imp = imputer.fit_transform(X_test)          # impute  
X_test_imp = pd.DataFrame(data=X_test_imp, index=[X_test.index], columns=[X_test.columns]) # turn created np.array back to pandas df

type(X_test_imp)

pandas.core.frame.DataFrame

In [97]:
# Preprocessing: scaling either standard or robust (removes median and scales data according to quantile range) 

scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_train_imp)
X_train_scaled = scaler.transform(X_train_imp)
# robust = preprocessing.RobustScaler()
# robust.fit_transform(X_train_imp)
# X_train_scaled = robust.transform(X_train_imp)

X_train_scaled = pd.DataFrame(data=X_train_scaled, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df
X_train_scaled

# nu ook voor X_TEST
scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_test_imp)
X_test_scaled = scaler.transform(X_test_imp)
# robust = preprocessing.RobustScaler()
# robust.fit_transform(X_train_imp)
# X_train_scaled = robust.transform(X_train_imp)

X_test_scaled = pd.DataFrame(data=X_test_scaled, index=[X_test.index], columns=[X_test.columns]) # turn created np.array back to pandas df
X_test_scaled

,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,logf_range_sigma1,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_027_S_0118_bl_0,-0.735248,0.938589,-0.844155,-1.622392,-1.238355,-1.080940,-1.025496,-0.359305,0.236178,-0.019094,-0.040156,0.157686,-0.057633,-0.744079,-0.474280,0.641413,1.295061,1.266156,-0.349895,-1.256962,-0.924771,0.156601,-1.127198,-0.680176,0.743479,-0.798649,-0.051558,0.079788,-0.444765,-0.341577,-0.325654,0.797529,0.199958,-0.524024,-0.713400,-0.991651,0.809522,0.432600,0.912775,0.292764,...,-0.274434,7.051893e-01,0.212208,-0.413213,-0.685015,-0.043610,-0.679746,-0.977601,-0.999559,-0.947209,-0.774055,1.219836,-0.482672,-0.962775,-0.764557,-0.863090,-0.685015,-0.043610,-0.679746,-0.977601,-0.999559,-0.947209,-0.774055,1.219836,-0.482672,-0.962775,-0.764557,-0.863090,-7.525135e-02,0.010681,-0.121494,0.358533,-0.436199,-1.062594e+00,-1.290145e-16,3.437830,5.298631e-01,0.497032,2.223647e-01,6.089666e-01
0_130_S_0956_bl_0,-0.088424,-2.217570,3.031007,-0.277663,0.454645,0.157144,1.815476,2.013206,-0.932239,-2.222792,-0.254722,-1.496615,-0.820496,0.628846,0.839382,1.432512,0.295050,0.491314,-1.026466,0.121789,0.277639,-0.514666,-0.660941,-0.215015,0.610062,-1.179056,-0.191856,0.059001,-1.002960,0.686551,-0.133716,-1.458993,-1.353376,-0.825102,-0.414357,-0.728141,0.839726,-0.125246,0.418525,-0.123722,...,-0.274433,-9.581575e-01,-0.570724,0.289179,-0.397996,-1.685739,1.008380,-0.662710,-0.261148,0.038333,0.772181,0.338234,-0.450144,-0.707216,0.067575,-0.648668,-0.397996,-1.685739,1.008380,-0.662710,-0.261148,0.038333,0.772181,0.338234,-0.450144,-0.707216,0.067575,-0.648668,-2.837075e-01,-2.286953,-1.014788,-1.267687,-0.909482,-7.336255e-01,6.928567e-

# We chose a RBF classifier, in the assignment there are 4 'forms' of RBF classifiers. 

# Option 1: use the standard RBF kernel from the scikit-learn SVM       #leek het niet te doen op de borst data set
clf = SVC(kernel='rbf', gamma=1)
clf.fit(X, Y)
y_pred = clf.predict(X)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))

# Option 2: Manually transform the feature space with an RBF kernel, sample a part of this, than apply a linear SVM
rbf_feature = RBFSampler(gamma=1, random_state=1)
clf = SVC(kernel='linear')
XR = rbf_feature.fit_transform(X)
clf.fit(XR, Y)
y_pred = clf.predict(XR)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))
    
# Option 3: supply a manually constructed kernel function to the SVM     #leek het niett te doen op de borst data set
clf = SVC(kernel=rbf_kernel)
clf.fit(X, Y)
y_pred = clf.predict(X)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))

# Option 4: precompute the RBF kernel                                    #leek het niet te doen op de borst data set
clf = SVC(kernel='precomputed')
rbf_kernel_computed = rbf_kernel(X)
clf.fit(rbf_kernel_computed, Y)
y_pred = clf.predict(rbf_kernel_computed)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))

In [0]:
# Set parameters
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-2],
                     'C': [1, 10]}]
                    
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

                    #in de exercises wordt voor gamma 'scale' of '1' gebruikt. 

scores = ['precision', 'recall']

In [99]:
#Grid search with cross-validation
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    ) 
    # cv is nu default, dus een kfold van 5! 
    clf.fit(X_train_scaled, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_train, clf.predict(X_train_scaled)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.787 (+/-0.079) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.766 (+/-0.068) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.772 (+/-0.054) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.760 (+/-0.070) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.82      0.75      0.78       239
           1       0.84      0.89      0.86       359

    accuracy                           0.83       598
   macro avg       0.83      0.82      0.82       598
weighted avg       0.83      0.83      0.83       598


# Tuning hyper-parameters for recall

Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel':

In [101]:
# Use the optimal parameters for training model
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

# kijken hoe we de beste parameters noemen zodat die hieronder makkelijk ingevoegd kan wordne, nu heb ik het nog handmatig gedaan.
clf = SVC(C=1, gamma=0.001, kernel='rbf')

model = clf.fit(X_train_scaled, y_train)
predictions = clf.predict(X_test_scaled)

print("score:")
print(model.score(X_test_scaled, y_test))




score:
0.8443579766536965
